In [106]:
from ollama import Client

client = Client(host='http://192.168.0.107:11434')

In [69]:
import os
import json

def read_local_json(name): 
    
    with open(f'./{name}.json') as f:
        data = json.load(f)

    return data

output_template = read_local_json('output_template')

criteria = output_template['Output']['evaluate_trials'][0]['criteria']
patient_profile = output_template['Output']['patient_profile']

VOTE_CNT = 3

In [103]:
def construct_msg(patient_des, criteria, criteria_type=""):
    messages = [
    {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
    },
    {
      "role": "system",
      "content": f'''
      You are responsible for analyzing patient profile and clinical trials criteria. Detailed factors and conditions in the criteria should be considered, do not make assumption when patient information is not enough. 
      First, You will be provided with the a clinical trial criterion. 
      Next, You will be provided with the patient profile. 
      ''',
        },
    {
      "role": "user",
      "content": f'''
        Here is the clinical trial {criteria_type} criterion:
        ${criteria}
        '''
    },
    {
      "role": "user",
      "content": f'''
        Here is the patient profile:
        ${patient_des}
        '''
    },
    {
      "role": "system",
      "content": f'''
      Now, Try to assess if the patient profile matches the criterion.
      ''',
    }
    ]

    inclusion_msg = {
      "role": "system",
      "content": f'''
      Finally, parse and return the result in JSON format:
        - match_result: one of the three possible values below indicating criterion matching result based on the patient profile
            * positive (The patient meets the requirement as stated in the inclusion criterion. The patient's profile aligns with the stated conditions or factors. This increases the chances of the patient being admitted into the clinical trial.)
            * neutral (The patient's profile neither clearly meets nor fails to meet the inclusion criterion. It's possible that some information is missing, not specific enough, or there may be some uncertainty about how well the patient aligns with the criterion. In this case, further information or tests might be required.)
            * negative (The patient does not meet the requirement as stated in the inclusion criterion. The patient's profile does not align with the conditions or factors stated and hence the patient is not suitable for the clinical trial.)
        - comment: comment on the evidence supporting one of the three values given for the criterion matching result based on the patient profile
      ''',
    }

    exclusion_msg = {
      "role": "system",
      "content": f'''
      Finally, parse and return the result in JSON format:
        - match_result: one of the three possible values below indicating criterion matching result based on the patient profile
            * positive (The patient meets the requirement as stated in the exclusion criterion, meaning the patient has certain conditions or factors that disqualify them from the clinical trial.)
            * neutral (The patient's profile neither clearly meets nor fails to meet the exclusion criterion. It's possible that some information is missing, not specific enough, or there may be some uncertainty about how well the patient aligns with the criterion. In this case, further information or tests might be required.)
            * negative (The patient does not meet any of the conditions or factors stated in the exclusion criterion. This means none of the disqualifying factors are present in the patient's profile, making them potentially suitable for the clinical trial.)
        - comment: comment on the evidence supporting one of the three values given for the criterion matching result based on the patient profile
      ''',
    }

    non_spec_msg = {
      "role": "user",
      "content": f'''
      Format the assessment result in the following JSON format:
        - match_result: one of the three possible values below indicating criterion matching result based on the patient profile
            * positive (meaning Patient profile contains the piece of information mentioned in the criterion and it meets this criterion)
            * neutral (meaning more information is needed from the Patient profile to determine if this patient meet the criterion)
            * negative (meaning Patient profile contains the piece of information mentioned in the criteria but it does not meet the criteria)
        - comment: comment on the evidence supporting one of the three values given for the criterion matching result based on the patient profile
      ''',
        }

    if criteria_type == "exclusion":
       messages.append(exclusion_msg)
    elif criteria_type == "inclusion":
       messages.append(inclusion_msg)
    else:
       messages.append(non_spec_msg)

    return messages

def lambda_handler(criterion, patient_profile):
  
  messages = construct_msg(patient_profile, criterion['criterion'], criterion['type'])

  result_llama = None
  while result_llama is None:
      try:
        response = client.chat(model='llama3:8b', messages=messages)
        result_llama = json.loads(response['message']['content'])
      except Exception as e:
          print(response['message']['content'])
          pass

  return {
          "id": criterion['id'],
          "type": criterion['type'],
          "criterion": criterion['criterion'],
          "match_result": result_llama['match_result'],
          "comment": result_llama['comment']
  }


In [104]:
final_output = []

print(f"###### Total: {len(criteria)}")
for vote_round in range(VOTE_CNT):
    print(f"##### VOTE Round {vote_round}#####")

    criteria_llama = []     

    
    for id, criterion in enumerate(criteria):
        
        print(id, end=';')

        if "match_result" in criterion.keys():

            result_criterion = lambda_handler(criterion, patient_profile)
            criteria_llama.append(result_criterion)
        else:
            criteria_llama.append(criterion)

        

    output_template_new = read_local_json('output_template')
    output_template_new['Output']['evaluate_trials'][0]['criteria'] = criteria_llama

    final_output.append(output_template_new)
    print(f"\n##### VOTE Round {vote_round}#####\n")
    
final_output = json.dumps(final_output)

###### Total: 34
##### VOTE Round 0#####
0;1;2;{
"match_result": "negative",
"comment": "The patient does not have symptomatic brain or leptomeningeal metastases, and their CNS symptoms are stable. Additionally, they do not require increasing doses of steroids and/or anti-seizure medications to manage CNS symptoms within two weeks prior to starting dacomitinib."
3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;{
"match_result": "positive",
"comment": "The patient's ECOG score is 0, which meets the inclusion criterion. Additionally, the patient has EGFR activating mutation (exon 19 deletion), which aligns with the criterion. All other factors and biomarkers also match, making it likely for the patient to meet the trial's inclusion criteria."
19;20;21;22;23;24;25;{
"match_result": "positive",
"comment": "The patient's ANC (16,000 cells/mm) meets the criterion of having an ANC ≥15,000 cells/mm3, which increases the chances of the patient being admitted into the clinical trial."
{
"match_result": 

In [105]:
print(final_output)

[{"ExecutionArn": "arn:aws:states:ap-east-1:201053736981:execution:clinmatch_match_patient_trial_evaluation_LLM_sub_process_no_vote:f6ead184-b7e3-4fa5-aee1-ae21f26f2301", "Input": {"patient_profile": "Patient with Non-Small Cell Lung Cancer, Adenocarcinoma. Disease occurred in lung. Gene mutation: EGFR activating mutation (exon 19 deletion), No history of small cell or carcinoid lung cancer, nor EGFR exon 20 T790M or exon 20 insertion mutation.. ECOG score is 0. Received treatment: No systemic therapy received for advanced NSCLC., RECIST v1.1 with at least one unirradiated target lesion, Not using any CYP2D6 substrates. Biomarkers are Platelets: 110,000 cells/mm, Creatinine: 35 mL/min, ALT (SGPT): Within normal limits, AST (SGOT): Within normal limits, Bilirubin, Total: 1.2 x ULN, ECOG: 0, ANC: 1600 cells/mm, Hemoglobin: 11.0 g/dL, Bilirubin: 1.2 x ULN. Other health status: Smoking status unknown.", "ranked_sql_trials": ["NCT04027647", "NCT04989322", "NCT05261399", "NCT05526755", "NCT0